In [1]:
from datasets import load_dataset

# Load the IMDB dataset
dataset = load_dataset("imdb")


/home/spandey7/miniconda3/envs/capstone/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer

# Load the tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the text
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 25000/25000 [00:05<00:00, 4578.96 examples/s]


In [3]:
from transformers import AutoModelForSequenceClassification

# Number of labels: 2 (positive and negative)
num_labels = 2

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Train the model
trainer.train()


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,0.276100,0.199232
2,0.125300,0.190123
3,0.080700,0.237019


TrainOutput(global_step=2346, training_loss=0.14814828139335195, metrics={'train_runtime': 2962.6513, 'train_samples_per_second': 25.315, 'train_steps_per_second': 0.792, 'total_flos': 1.9733329152e+16, 'train_loss': 0.14814828139335195, 'epoch': 3.0})

In [5]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Update the Trainer instance with the new compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

# Perform the evaluation again with the new metrics
evaluation_results = trainer.evaluate()
print(evaluation_results)


Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


{'eval_loss': 0.23701944947242737, 'eval_accuracy': 0.9404, 'eval_f1': 0.9409152192878102, 'eval_precision': 0.9328510772133983, 'eval_recall': 0.94912, 'eval_runtime': 241.5764, 'eval_samples_per_second': 103.487, 'eval_steps_per_second': 3.237}


In [6]:
#save the model
trainer.save_model("./results")